In [1]:
# Importing modules
import os
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

In [2]:
# Path to train, val and test folder
train_dir = './train/'
val_dir = './val/'
test_dir = './test/'

In [3]:
# Transformation
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

# Train dataset
train_dataset = datasets.ImageFolder(train_dir, transform)

# Validation dataset
val_dataset = datasets.ImageFolder(val_dir, transform)

# Test datasets
test_dataset = datasets.ImageFolder(test_dir, transform)

# Train dataloader
train_dataloader = DataLoader(train_dataset, batch_size=128)

# Val dataloader
val_dataloader = DataLoader(val_dataset, batch_size=128)

# Test dataloader
test_dataloader = DataLoader(test_dataset, batch_size=128)

In [4]:
# SimpleNet Model
class SimpleNet(nn.Module):

  def __init__(self):
    super(SimpleNet, self).__init__()
    self.fc1 = nn.Linear(12288, 84)
    self.fc2 = nn.Linear(84, 50)
    self.fc3 = nn.Linear(50, len(os.listdir(train_dir)))

  def forward(self, x):
    x = torch.flatten(x, 1)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [5]:
# Model
model = SimpleNet()

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Loading model to device
model = model.to(device)

In [7]:
# Training
for epoch in range(20):

    # Training loop
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    average_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1} | Validation Loss: {average_val_loss}")

Epoch 1 | Validation Loss: 3.953335616964614
Epoch 2 | Validation Loss: 1.286982897669077
Epoch 3 | Validation Loss: 0.7420513987541199
Epoch 4 | Validation Loss: 0.5281069993972778
Epoch 5 | Validation Loss: 0.5656817883253098
Epoch 6 | Validation Loss: 0.4726168870925903
Epoch 7 | Validation Loss: 0.45422913134098053
Epoch 8 | Validation Loss: 0.42337810099124906
Epoch 9 | Validation Loss: 0.3951799660921097
Epoch 10 | Validation Loss: 0.3979069232940674
Epoch 11 | Validation Loss: 0.34705943763256075
Epoch 12 | Validation Loss: 0.33448357582092286
Epoch 13 | Validation Loss: 0.3315883904695511
Epoch 14 | Validation Loss: 0.3041655898094177
Epoch 15 | Validation Loss: 0.3001324951648712
Epoch 16 | Validation Loss: 0.29386927783489225
Epoch 17 | Validation Loss: 0.286748132109642
Epoch 18 | Validation Loss: 0.28279435336589814
Epoch 19 | Validation Loss: 0.27896781265735626
Epoch 20 | Validation Loss: 0.2756479024887085


In [8]:
y_pred = []
y_test = []

# Testing loop
model.eval()
with torch.no_grad():
    for batch in test_dataloader:
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        pred = torch.argmax(outputs, dim=1)
        y_pred.extend(pred.cpu().numpy())
        y_test.extend(targets.cpu().numpy())

In [9]:
# Evaluation metric
report = classification_report(y_test, y_pred, target_names=test_dataset.classes)
print(report)

              precision    recall  f1-score   support

    basophil       0.93      0.77      0.84       100
  lymphocyte       0.80      0.94      0.87       100

    accuracy                           0.85       200
   macro avg       0.87      0.85      0.85       200
weighted avg       0.87      0.85      0.85       200

